In [1]:
# import required packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [18]:
# load csv data for input and output

# 7 columns each for joint angles, angular velocities and angular acceleraions
uVec = pd.read_csv("Sarcos.csv", usecols=np.arange(21),  header=None,
                  names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 
                           'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7'])
# 7 columns for joint torques
yVec = pd.read_csv("Sarcos.csv", usecols=np.arange(21, 28, 1), header=None, 
                  names = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7'])

# to numpy array
u = uVec.values[:, :, np.newaxis]
y = yVec.values[:, :, np.newaxis]

# generate layer
eta = 0.9
layer_size = 21
input_size = u.shape[1]
Layer = SelfOrganizedLayer(layer_size, input_size, eta)

In [19]:
# test layer
Layer.layerReset()
for i in range(len(u)):
    unorm, s = Layer.updateOutput(u[i])
    Layer.updateWeightMatrix(unorm, s)

s

array([[ 0.96648891],
       [ 0.31208504],
       [ 0.12659149],
       [ 0.06573441],
       [ 0.06350969],
       [-0.00244234],
       [ 0.02902677],
       [-0.01370289],
       [ 0.0392321 ],
       [-0.03443209],
       [ 0.03071527],
       [ 0.04533702],
       [-0.02833523],
       [ 0.04425619],
       [ 0.03191195],
       [ 0.00727233],
       [-0.01717282],
       [-0.03227183],
       [ 0.04422893],
       [-0.03018723],
       [-0.01583407]])

In [17]:

print(Layer.W)

[[-4.32443022e-01 -1.31838976e-01  1.14669472e-01  2.41195245e-01
   2.99360389e-01 -3.50766260e-01 -9.03282677e-02  2.33654894e-01
   4.39574384e-01 -1.56294999e-01 -4.98832543e-01 -5.46375212e-01
   2.71644328e-01  4.36961380e-01  1.60498060e-01  2.06196603e-01
  -2.27729839e-01 -2.71014394e-01 -2.39015369e-01  3.78878035e-01
   4.11765237e-01]
 [ 2.53736005e-02  9.77854085e-03  6.17246688e-03 -1.29020947e-02
  -1.31147517e-02  1.40681055e-02 -2.67294872e-03  5.36411955e-02
   1.08141460e-01 -4.72313917e-02 -1.26498259e-01 -1.42981344e-01
   7.34373689e-02  1.20505173e-01 -4.60987841e-03  4.36091349e-03
  -3.26669117e-02 -2.00942439e-02  3.27476373e-02  7.02099600e-02
   5.42816673e-02]
 [ 6.99359547e-03  5.94007499e-03 -2.11935489e-03 -7.40093869e-03
  -9.86915138e-03  2.58683952e-03  6.18087499e-03  2.61596845e-02
   4.65173306e-02 -1.90184749e-02 -5.26531833e-02 -6.47024654e-02
   2.84211114e-02  4.53089741e-02  4.85512110e-04 -1.64186626e-03
  -1.81778809e-03 -3.32795692e-03  1.8

In [6]:
class SelfOrganizedLayer:
    '''''Self organized layer for Neural Networks using the Generalized Hebbian Learning (GHL) algorithm to update weights. 
    Functions:
        * __init__ : initilizing the network layer, creating a random weight matrix and related variables
        * updateOutput: calcualtes the output for a given input vector. The input is centerd within the function
        * updateWeightMatrix: updates the weight accoring to the GHL algorithm'''''
    
    def __init__(self, layer_size, input_size, eta):
        '''layer_size - number of neurons
        input_size - number of inputs
        eta - learning rate'''
        
        # initalize W with random weights
        self.W = np.random.normal(size = (layer_size, input_size))
        self.W = self.W/np.sum(self.W)
        
        # save learning rate
        self.eta = eta
        
        # initalize input sum and mean
        self.layerReset()
        
    def layerReset(self):
        self.uMean = 0 
        self.uAbs = 0.00001
        self.i = 0
    
    def updateOutput(self, u):
        '''u - network input vector'''
        
        # normalize and center input
        self.uMean = self.uMean + (u-self.uMean)/(self.i+1)
        uCenter = u - self.uMean
        self.uAbs = np.maximum(self.uAbs, np.absolute(uCenter))
        uNormCenter = uCenter/self.uAbs
        
        # update index
        self.i += 1
        
        # calculate output
        s = np.tanh(self.W@uNormCenter)
        
        return (uNormCenter, s)
    
    def updateWeightMatrix(self, u, s):
        '''u - network input vector
        s - network output'''
        # transpose
        uT = np.transpose(u)
        sT = np.transpose(s)
        
        # calculate GHL update 
        triang = np.tril(s@sT);
        dW = self.eta*(s@uT - triang@self.W);
        
        # update W matrix
        self.W += dW
        